In [1]:
# Code to create a model used to predict whether the individual patient has an allergy
# Use different notebook to load the model and return a prediction

import os
os.environ["KERAS_BACKEND"] = "tensorflow"
import tensorflow
import keras

import numpy as np
np.random.seed(123)

2024-03-05 19:13:30.866718: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
import matplotlib
from keras.models import * 
from keras.layers import *
from keras.optimizers import RMSprop
import pandas as pd

# Import both datasets, change to local path when running
patients = pd.read_excel(r"/Users/ellagodfrey/Desktop/492/patients.xlsx", sheet_name="Level2_AI_Patient Traits")

allergies = pd.read_excel(r"/Users/ellagodfrey/Desktop/492/patients.xlsx", sheet_name="Level1_Patient Allergens")

In [3]:
# Confirm that patient sheet imported correctly
# Comment below line before committing
# patients['SkinConditions'].head()

In [4]:
# Confirm allergy sheet imported correctly
# Comment line before committing
# allergies.head()

In [5]:
# Merge columns by ID if needed
patientAllergies = patients.merge(allergies, on = "SFM Id")
# Comment line before committing
# patientAllergies.head()

In [6]:
# Drop ID and location columns from dataframe
patientsTrimmed = patients.drop(['SFM Id', 'City', 'State', 'Country'], axis = 1)

In [7]:
# One-hot encode Gender column
patientsTrimmed['Gender'] = pd.Categorical(patientsTrimmed['Gender'].str.strip())
gender_onehot = pd.get_dummies(patientsTrimmed['Gender'], prefix = "Gender",
                                    prefix_sep = "-", dtype = int)
patientsTrimmed = patientsTrimmed.drop('Gender', axis = 1)
patientsTrimmed = patientsTrimmed.join(gender_onehot)

In [8]:
# One-hot encode SkinTone column
patientsTrimmed['SkinTone'] = pd.Categorical(patientsTrimmed['SkinTone'].str.strip())
skintone_onehot = pd.get_dummies(patientsTrimmed['SkinTone'], prefix = "SkinTone",
                                    prefix_sep = "-", dtype = int)
patientsTrimmed = patientsTrimmed.drop('SkinTone', axis = 1)
patientsTrimmed = patientsTrimmed.join(skintone_onehot)

In [9]:
# One-hot encode FitzPatrickSkinPhotoType column
patientsTrimmed['FitzpatrickSkinPhotoType'] = pd.Categorical(patientsTrimmed['FitzpatrickSkinPhotoType'].str.strip())
# Dropping first here since it is a blank variable in the column
fitzpatrick_onehot = pd.get_dummies(patientsTrimmed['FitzpatrickSkinPhotoType'], prefix = "Fitzpatrick",
                                    prefix_sep = "-", drop_first = True, dtype = int)
patientsTrimmed = patientsTrimmed.drop('FitzpatrickSkinPhotoType', axis = 1)
patientsTrimmed = patientsTrimmed.join(fitzpatrick_onehot)

In [10]:
# Switching to TextVectorization (Tokenizer is deprecated)
from keras.layers import TextVectorization
# Replace commas with whitespace
patientsTrimmed['SkinConditions'] = patientsTrimmed['SkinConditions'].str.replace(',', ' ')
# Set the max length based on whitespace characters
max_len = patientsTrimmed['SkinConditions'].str.count(' ').max()
# Create TextVectorization object, separating on whitespace and using the max_len from earlier
vectorizer = TextVectorization(split = 'whitespace', output_sequence_length = max_len)

In [11]:
# Adapt using the column we want to convert
vectorizer.adapt(patientsTrimmed['SkinConditions'].values)
# Reset the column after converting values to vector and placing in array
skinConditions = vectorizer(patientsTrimmed['SkinConditions']).numpy()
patientsTrimmed = patientsTrimmed.drop('SkinConditions', axis = 1)

In [12]:
patientsArr = patientsTrimmed.values
input_data = np.concatenate((patientsArr, skinConditions), axis = 1)

In [13]:
''' Old code for SkinConditions conversion - leaving in case we need later
# Code to preprocess the SkinConditions column, tokenizing should be more recognizable to keras than regular text
## Delete if this idea does not work
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

# Flatten entries into actual lists
patientsTrimmed['SkinConditions'] = patientsTrimmed['SkinConditions'].apply(lambda x: ' '.join(x))
# Tokenize the characters of the columns
tokenizer = Tokenizer()
tokenizer.fit_on_texts(patientsTrimmed['SkinConditions'])
# Create and pad sequences of the tokens
patientsTrimmed['SkinConditions'] = tokenizer.texts_to_sequences(patientsTrimmed['SkinConditions'])
max_length = max(len(seq) for seq in patientsTrimmed['SkinConditions'])
testVar = pad_sequences(patientsTrimmed['SkinConditions'], maxlen = max_length, padding = "post")
patientsTrimmed['SkinConditions'] = testVar.tolist()
'''

' Old code for SkinConditions conversion - leaving in case we need later\n# Code to preprocess the SkinConditions column, tokenizing should be more recognizable to keras than regular text\n## Delete if this idea does not work\nfrom keras.preprocessing.text import Tokenizer\nfrom keras.preprocessing.sequence import pad_sequences\n\n# Flatten entries into actual lists\npatientsTrimmed[\'SkinConditions\'] = patientsTrimmed[\'SkinConditions\'].apply(lambda x: \' \'.join(x))\n# Tokenize the characters of the columns\ntokenizer = Tokenizer()\ntokenizer.fit_on_texts(patientsTrimmed[\'SkinConditions\'])\n# Create and pad sequences of the tokens\npatientsTrimmed[\'SkinConditions\'] = tokenizer.texts_to_sequences(patientsTrimmed[\'SkinConditions\'])\nmax_length = max(len(seq) for seq in patientsTrimmed[\'SkinConditions\'])\ntestVar = pad_sequences(patientsTrimmed[\'SkinConditions\'], maxlen = max_length, padding = "post")\npatientsTrimmed[\'SkinConditions\'] = testVar.tolist()\n'

In [14]:
# Drop ID column for preprocessing - ID should have no effect on prediction
allergiesNoId = allergies.drop('SFM Id', axis = 1)
# Remove all non-digit characters, then replace empty cells with NaN
allergiesNoId = allergiesNoId.replace(r'\D+', '', regex = True).replace('', np.nan)
# Set all NaN cells to 0
allergiesNoId = allergiesNoId.fillna(0)
# Convert entire dataframe to integer
allergiesNoId = allergiesNoId.astype(int)

In [15]:
allergiesNoId['AllergiesList'] = allergiesNoId.astype(str).apply(' '.join, axis=1)
allergiesNoId['AllergiesList'] = allergiesNoId['AllergiesList'].str.split()

In [16]:
from sklearn.preprocessing import MultiLabelBinarizer
# Create MultiLabelBinarizer object
mlb = MultiLabelBinarizer()

allergiesArr = np.array(allergiesNoId['AllergiesList'])
# Multi-hot encode data
allergiesArray = mlb.fit_transform(allergiesArr)

In [17]:
allergiesNew = mlb.inverse_transform(allergiesArray)

In [18]:
allergiesDF = pd.DataFrame(mlb.transform(allergiesArr), columns = mlb.classes_)
allergiesDF.dtypes

0         int64
100612    int64
100613    int64
100702    int64
100857    int64
          ...  
9804      int64
98288     int64
99100     int64
9926      int64
99356     int64
Length: 731, dtype: object

In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import multilabel_confusion_matrix, precision_score, recall_score, f1_score
from keras.layers import Dense
import keras
import numpy as np

cv = KFold(n_splits=5, shuffle=True, random_state=7869)

fold_count = 1
# Train the model for each split
# Define the model inside the for loop
for train, test in cv.split(input_data, allergiesArray):

    n_classes = 731

    # Input layer
    input_shape = (40,)
    inputs = keras.Input(input_shape)

    # Hidden layers
    x = Dense(256, activation='sigmoid')(inputs)
    x = Dense(512, activation='sigmoid')(x)

    # Output layer - use multilabel classification
    predictions = Dense(n_classes, activation='sigmoid')(x)

    model = keras.Model(inputs=inputs, outputs=predictions)
    model.compile(loss=keras.losses.BinaryFocalCrossentropy(apply_class_balancing=True, alpha=0.35, gamma=14),
                  optimizer=keras.optimizers.Adam(learning_rate=0.1),
                  metrics=[keras.metrics.Precision(), keras.metrics.Recall()])

    print("Running fold #: ", fold_count)

    fold_train_x = input_data[train]

    history = model.fit(
        fold_train_x, allergiesArray[train],
        epochs=20,
        verbose=1,
        validation_split=0.2
    )

    fold_test_x = input_data[test]
    y_true = allergiesArray[test]

    # Predict probabilities
    probs = model.predict(fold_test_x, verbose=1)

    # Dynamic threshold optimization based on F1 score
    best_f1 = 0
    optimal_threshold = 0
    for t in np.arange(0.1, 1, 0.1):
        y_pred = (probs > t).astype(int)
        f1 = f1_score(y_true, y_pred, average='weighted', zero_division=1)
        if f1 > best_f1:
            best_f1 = f1
            optimal_threshold = t

    print("Best F1 score:", best_f1)
    print("Optimal threshold:", optimal_threshold)

    # Apply threshold
    y_pred = (probs > optimal_threshold).astype(int)

    # Compute precision and recall
    precision = precision_score(y_true, y_pred, average='weighted', zero_division=1)
    recall = recall_score(y_true, y_pred, average='weighted', zero_division=1)

    print("Precision:", precision)
    print("Recall:", recall)

    # Compute multilabel confusion matrix
    matrix = multilabel_confusion_matrix(y_true, y_pred)
    print(matrix)

    fold_count += 1


Running fold #:  1
Epoch 1/20
198/198 [==============================] - 6s 20ms/step - loss: 0.0060 - precision: 0.0661 - recall: 0.2249 - val_loss: 0.0069 - val_precision: 0.1082 - val_recall: 0.1673
Epoch 2/20
198/198 [==============================] - 4s 21ms/step - loss: 0.0032 - precision: 0.1365 - recall: 0.1416 - val_loss: 0.0024 - val_precision: 0.3525 - val_recall: 0.1487
Epoch 3/20
198/198 [==============================] - 5s 27ms/step - loss: 0.0025 - precision: 0.2441 - recall: 0.1394 - val_loss: 0.0019 - val_precision: 0.1301 - val_recall: 0.1463
Epoch 4/20
198/198 [==============================] - 5s 24ms/step - loss: 0.0016 - precision: 0.3003 - recall: 0.1343 - val_loss: 0.0019 - val_precision: 0.2505 - val_recall: 0.1409
Epoch 5/20
198/198 [==============================] - 4s 19ms/step - loss: 0.0016 - precision: 0.3398 - recall: 0.1335 - val_loss: 0.0026 - val_precision: 1.0000 - val_recall: 0.1406
Epoch 6/20
198/198 [==============================] - 4s 20ms/step

Epoch 1/20
198/198 [==============================] - 8s 26ms/step - loss: 0.0101 - precision_2: 0.0681 - recall_2: 0.2194 - val_loss: 0.0075 - val_precision_2: 0.1439 - val_recall_2: 0.1580
Epoch 2/20
198/198 [==============================] - 4s 23ms/step - loss: 0.0046 - precision_2: 0.1417 - recall_2: 0.1458 - val_loss: 0.0046 - val_precision_2: 0.1241 - val_recall_2: 0.1533
Epoch 3/20
198/198 [==============================] - 4s 19ms/step - loss: 0.0028 - precision_2: 0.2530 - recall_2: 0.1351 - val_loss: 0.0039 - val_precision_2: 0.2511 - val_recall_2: 0.1379
Epoch 4/20
198/198 [==============================] - 3s 15ms/step - loss: 0.0022 - precision_2: 0.3298 - recall_2: 0.1333 - val_loss: 0.0030 - val_precision_2: 0.3346 - val_recall_2: 0.1378
Epoch 5/20
198/198 [==============================] - 5s 26ms/step - loss: 0.0023 - precision_2: 0.3823 - recall_2: 0.1335 - val_loss: 0.0035 - val_precision_2: 0.5000 - val_recall_2: 0.1372
Epoch 6/20
198/198 [=========================

In [ ]:
# TODO: Save Keras model as separate file